In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math
import scipy.constants as sci
plt.rcParams["font.family"] = "Arial"
DPI = 220
Z = 18
import matplotlib as mpl
from matplotlib import cm
mpl.rcParams['axes.linewidth'] = 1.4
import os
import imageio

In [2]:
def read_interdensity(fname):
    f = open(fname, "r")
    fl = f.readlines()
    f.close()
    
    ids = []
    data = []
    tajada = []
    for line in fl:
        if "#" not in line:
            if "$" in line and tajada != []:
                data.append(np.array(tajada, dtype='float'))
                tajada = []
            elif "$" not in line:
                tajada.append(np.array(line.split(), dtype='float'))
            if "$" in line:
                ids.append(float(line.split()[3]))
    data.append(np.array(tajada, dtype='float'))
    data = np.array(data)
    data = data/0.001 #for a grid size of 1A, 0.001 converts charge into charge density
    return data, ids        

In [3]:
nano_labels = ["N40"]
ion_labels = ["I10", "I20"]
ion_values = [33, 66]
fini=23
ffin=23
frames = np.linspace(fini, ffin, ffin-fini+1, dtype='int')

In [ ]:
cmap='seismic'
for nano_label in nano_labels:
    #os.mkdir("T2/INTERDENSITY/{}".format(nano_label))
    for ion_label in ion_labels:
        #os.mkdir("T2/INTERDENSITY/{}/{}".format(nano_label, ion_label))
        for frame in frames:
            print("{}/{}/F{}".format(nano_label, ion_label, frame))
            #os.mkdir("T2/INTERDENSITY/{}/{}/Frame{}".format(nano_label, ion_label, frame))
            data, ids = read_interdensity("T2/INTERDENSITY/T2-{}-{}_L_MD{}_interdensity.sfu".format(nano_label, ion_label, frame))
            n_slices = len(data)
            n_res = len(data[0])
            
            images = []
            for i in range(n_slices):
                fname = "T2/INTERDENSITY/{}/{}/Frame{}/img{}.png".format(nano_label, ion_label, frame, i)
                fig = plt.figure(figsize=(6,6))
                ax = plt.axes()
                ax.tick_params(labelsize=Z)
                ax.set_xlabel("CV2 (nm)", fontsize=Z)
                ax.set_ylabel("CV3 (nm)", fontsize=Z)
                ax.set_title(r"S = " + r"$\bf{" + "{:>6.2f}".format(ids[i]) + r"}$ nm", fontsize=Z-2)
                ax.set_xticks(np.linspace(0, n_res-1, 5))
                ax.set_xticklabels([-1.50, -0.75, 0.00, 0.75, 1.50])
                ax.set_yticks(np.linspace(0, n_res-1, 5))
                ax.set_yticklabels([-1.50, -0.75, 0.00, 0.75, 1.50])
                cax = ax.imshow(data[i], cmap=cmap, interpolation='bilinear', vmin=-1000, vmax=1000)

                a = plt.axes([0.95, 0.125, 0.05, 0.76])
                cbar = fig.colorbar(cax, ax=a, cax=a, ticks=[-1000, -750, -500, -250, 0, 250, 500, 750, 1000])
                cbar.ax.tick_params(labelsize=Z)
                cbar.ax.set_ylabel(r"Surface charge density ($e$ $nm^{-2}$)", fontsize=Z)
                plt.savefig(fname, format='png', dpi=DPI, bbox_extra_artists=(a,), bbox_inches='tight')
                plt.close()
                images.append(imageio.imread(fname)) 
            imageio.mimsave('T2/INTERDENSITY/{}/{}/Frame{}.gif'.format(nano_label, ion_label, frame), images)

N40/I10/F23
N40/I20/F23
